# Pipeline Research
when developing a pipeline, I use this notebook to play around with the data and see what I can do with it.

In [7]:
from datasets.documents_dataset import DocumentsDataset
from datasets.stock_dataset import StockMeta
from pipeline.document_filterer import DefaultFilterer

doc_dataset = DocumentsDataset(documents_csv_path="./organized_data/documents.csv")
filterer = DefaultFilterer()
stock_meta = StockMeta(stock_meta_path="./organized_data/stock_metadata.csv")

In [3]:
stock_name = stock_meta.stock_names[20]
stock = stock_meta.get_stock_by_name(stock_name)
filtered_docs = filterer.filter_documents(doc_dataset, stock)
filtered_docs

TypeError: argument of type 'float' is not iterable

In [5]:
filtered_documents = []
for document in doc_dataset:
    if stock_name in document.title or stock_name in document.content:
        filtered_documents.append(document)

TypeError: argument of type 'float' is not iterable

In [11]:
import pandas as pd

df = pd.read_csv("./organized_data/documents.csv")

In [12]:
df.isnull().sum()

p_type                 0
s_name                 0
s_area_name            0
post_time              0
title                  0
author             26826
content             9896
content_type     1020442
comment_count    1020442
dtype: int64